# Face Recognition in Low Resolution

Actually identifies the faces. Before running, be sure you have encoded the faces. If you havne't, go back to `step_1_encode_faces` notebook

## Step 1: Import required libraries & initialize functions

Be sure to run these first.

In [24]:
from matplotlib import pyplot as plt

# USAGE
# python recognize_faces_image.py --encodings encodings.pickle --image examples/example_01.png 

# import the necessary packages
import face_recognition
import pickle
import cv2
from imutils import paths # to get paths
import os
import numpy
import time
import pandas as pd

# for cross-functions
from cv2 import dnn_superres
from matplotlib import pyplot as plt
from PIL import Image
import warnings

def show_image(title, img):
      show = img[:,:,::-1] # needed to reverse colour properly. select all the elements on the first axis, all the elements on the seconds, and then the reverse of the elements on the third axis.
      plt.imshow(show)
      plt.title(title)
      plt.show()

def optimizeImage(imagePath, modelPath):
    # Create an SR object
    sr = dnn_superres.DnnSuperResImpl_create()
    
    # Get model name, get text between '/' & '_'
    modelName = modelPath.split('/')[1].split('_')[0].lower()

    # Get scale
    scale = int(modelPath.split('/')[1].split('_')[1].split('.')[0][1])

    # Display the name and scale
    # print("Model name: "+ modelName)
    # print("Model scale: " + str(modelScale))

    # Read image
    image = cv2.imread(imagePath)
    
    # Read the desired model
    sr.readModel(modelPath)

    # Set the desired model and scale to get correct pre- and post-processing
    sr.setModel(modelName, scale)

    # Upscale the image
    result = sr.upsample(image)

    # Save the image (optional)
    #cv2.imwrite("./examples/upscaled-fsrcnn_x4.jpg", result)
    
    return result

## for use in bicubic START
def scaleImage(imagePath, factor, algorithm):
    # Read image
    image = Image.open(imagePath)
    imgSize = (image.width * factor, image.height * factor)

    # Upscale the image
    result = image.resize(imgSize, algorithm)

    return result

# for non-opencv use
def display_image(title, img):
      plt.imshow(img)
      plt.title(title)
      plt.show()
## for use in bicubic END

# detection_mode: # variable for setting detection method, either: 'scale', 'ai', or anything else for 'direct'
def recognize_face(detection_mode, dataset):
    encodings = "encodings.pickle"
    detection_method = "hog"
    imagePaths = list(paths.list_images(dataset))
    
    # benchmark time
    start_time = time.time()
    # variables for detection accuracy
    identify_correct = 0
    identify_wrong = 0
    identify_none = 0
    count = 0

    # load the known faces and embeddings
    print("[INFO] loading encodings...")
    data = pickle.loads(open(encodings, "rb").read())
    
    if (len(imagePaths) == 0):
        print("[WARN] No image in ", dataset, "directory detected, maybe add some pictures first?")
    
    for imagePath in imagePaths:
        count += 1
        print("Detecting face ", count, " out of ", len(imagePaths))
        if (detection_mode == "scale"):
            pil_img = scaleImage(imagePath, 4, Image.BICUBIC)
            image = cv2.cvtColor(numpy.array(pil_img), cv2.COLOR_RGB2BGR)
            # print("scaling")
        elif (detection_mode == "ai"):
            image = optimizeImage(imagePath, "models/EDSR_x4.pb")
            # print("ai")
        else:
            pil_img = scaleImage(imagePath, 4, Image.NEAREST)
            image = cv2.cvtColor(numpy.array(pil_img), cv2.COLOR_RGB2BGR)
            # print("direct")
            
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # detect the (x, y)-coordinates of the bounding boxes corresponding
        # to each face in the input image, then compute the facial embeddings
        # for each face
        print("[INFO] recognizing faces in ", imagePath)
        boxes = face_recognition.face_locations(rgb,
            model=detection_method)
        encodings = face_recognition.face_encodings(rgb, boxes)

        # initialize the list of names for each face detected
        names = []

        # loop over the facial embeddings
        for encoding in encodings:
            # attempt to match each face in the input image to our known
            # encodings
            matches = face_recognition.compare_faces(data["encodings"],
                encoding)
            name = "Unknown"
            # check to see if we have found a match
            if True in matches:
                # find the indexes of all matched faces then initialize a
                # dictionary to count the total number of times each face
                # was matched
                matchedIdxs = [i for (i, b) in enumerate(matches) if b]
                counts = {}

                # loop over the matched indexes and maintain a count for
                # each recognized face
                for i in matchedIdxs:
                    name = data["names"][i]
                    counts[name] = counts.get(name, 0) + 1

                # determine the recognized face with the largest number of
                # votes (note: in the event of an unlikely tie Python will
                # select first entry in the dictionary)
                name = max(counts, key=counts.get)

            # update the list of names
            names.append(name)

        # loop over the recognized faces
        for ((top, right, bottom, left), name) in zip(boxes, names):
            # draw the predicted face name on the image
            cv2.rectangle(image, (left, top), (right, bottom), (0, 0, 255), 1)
            y = top - 15 if top - 15 > 15 else top + 15
            cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
                0.25, (0, 255, 0), 1)

        # show the output image
        # cv2.namedWindow("Image", cv2.WINDOW_NORMAL) 
        # cv2.imshow("Image", image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        correctName = [imagePath.split(os.path.sep)[1]] # 0 is root directory, 1 is subfolder (name), 2 is image name
        if names == correctName:
            print("Correct face recognized")
            identify_correct += 1
        else:
            print("Folder name is: ", correctName, ". However, face detected is:", names)
            if names: #implicit booliness
                identify_wrong += 1
            else:
                identify_none += 1
        img2 = image[:,:,::-1] # needed to reverse colour properly. select all the elements on the first axis, all the elements on the seconds, and then the reverse of the elements on the third axis.
        plt.imshow(img2)
        plt.title(imagePath)
        plt.show()
    
    totalPictures = len(imagePaths)
    if (totalPictures > 0):
        print("Summary")
        print("==========================")
        print("Total pictures", totalPictures)
        print("Faces identified correctly: ", identify_correct)
        print("Faces identified wrongly: ", identify_wrong)
        print("Faces not identified: ", identify_none)
        print("Absolute Accuracy (correct / [correct + incorrect + not detected]) * 100%: ")
        print(round(identify_correct / (identify_correct + identify_wrong + identify_none), 2) * 100, "%")
        print("Detection Accuracy [(correct + incorrect) / total] * 100%: ")
        print(round(((identify_correct + identify_wrong) / totalPictures) * 100, 2), "%")
        print("END, time taken: ", round((time.time() - start_time), 2), "seconds")
        print("Avg. secs. per picture: ", round((time.time() - start_time) / totalPictures, 2), "seconds")
        print("==========================")
    
print("Ready!")

Ready!


## Step 2: Pick one of the options below for face identification

Click inside the desired code block, then press "CTRL" + "Enter" key to run it

### Option 1: Direct Face Identification

- Direct using nearest neighbour.
- Image is scaled up 4 times using the `Image.NEAREST` option
- Very fast

In [1]:
print("Face identification: Direct using nearest neighbour")
print("=========================================================")
recognize_face("direct", "examples")

Face identification: Direct using nearest neighbour


NameError: name 'recognize_face' is not defined

In [2]:
print("Face identification: Scalin with bicubic")
print("=========================================================")
recognize_face("scale", "examples")

Face identification: Scalin with bicubic


NameError: name 'recognize_face' is not defined

### Option 2: Scaling

- Uses bicubic scaling to upscale the image 4x
- Typically results in much better accuracy compared to option 1
- Fast

In [3]:
print("Face identification: Scalin with bicubic")
print("=========================================================")
recognize_face("scale", "examples")

Face identification: Scalin with bicubic


NameError: name 'recognize_face' is not defined

### Option 3: Recognize using AI (EDSR x4)

- Try to improve the image using EDSR super-resolution. 
- Typically, results in better detection accuracy than option 2. Note that "better" is subjective. Sometimes, it may miss more faces, but the false-positive also reduces (which means its technically better, because not detecting is better than detecting wrongly).
- Very slow

In [4]:
print("Face identification: Scalin with EDSR")
print("=========================================================")
recognize_face("ai", "examples")

Face identification: Scalin with EDSR


NameError: name 'recognize_face' is not defined